In [ ]:
import sys
sys.path.append("/Users/user/question-retrieval-KIPerWeb/")
from utils import *
import pandas as pd
import re
from sentence_transformers import SentenceTransformer
import json
from nltk.stem.cistem import Cistem
from nltk.corpus import stopwords
from ranx import Qrels, evaluate, Run
import swifter
import json
import requests
from sklearn.preprocessing import LabelEncoder
from trectools import TrecPoolMaker, TrecRun
from tqdm import tqdm
from collections import Counter
import spacy

from rank_bm25 import BM25Okapi
from pprint import pprint




In [ ]:
# Read preprocessed data
qb = pd.read_csv("/Users/user/question-retrieval-KIPerWeb/testbeds/samples/qb_stratified-sample.csv").fillna("N/A")
# df = df.query("topic_label_de!='unbekannt'").reset_index()
qb = qb.drop(columns=['docid'])
qb.rename(columns={"category":"topic_label_de_fixed"}, inplace=True)
qb['docid'] = qb.index

qb['content'] = qb['content'].apply(lambda x:remove_stop_words(x, 'german'))


# Preprocessing
# Models for preprocessing
nlp = spacy.load("de_core_news_sm")
tagger_de = ht.HanoverTagger('morphmodel_ger.pgz')

qb['content'] = qb['content'].apply(lambda x : preprocess_documents(x, tagger_de))
qb

In [ ]:
print(list(qb.columns))

## BM25 Model


In [ ]:
# from gensim.summarization.bm25 import BM25
from gensim.summarization.bm25 import BM25
from pprint import pprint
    
def BM25_ranker(query, df, k, **kwargs):

    df['temp_index'] = df.index
    corpus = df['content'].tolist()
    bm25 = BM25(corpus)
    scores = bm25.get_scores(query)
    indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:k]
    scores = bm25.get_scores(query)[:k]
    results_df = df.query("temp_index in @indices")[["docid", "onlinetest_title", "question_type_id", "question_type_name", "answer_type_id", "answer_type_name", "text", "correct_answers_temp", "variable", "points", "answers", "source", "level_difficulty", "topic_label_de_fixed", "related_topics"]]
    question_ids = results_df['docid'].tolist()
    results = list(results_df.T.to_dict().values())
    for i, j in zip(results, scores):
        i['BM25_score'] = j
    for e in results:
        e['query'] = query
    if 'run_path' in kwargs:
        save_to_json(results, kwargs['run_path'], model_name='BM25')
    else:
        pass
    return results

BM25_ranker('konzentrationsproblem', qb, 5, run_path='/Users/user/Downloads/run_updated_vbm25.json')


## Run Queries Against Search Indices

In [ ]:

queries_file = pd.read_csv("/Users/user/question-retrieval-KIPerWeb/testbeds/queries_experiments/queries/queries.csv")
queries_file['queries'] = queries_file['queries'].apply(clean_text)



print("-> Number of queries:",len(queries_file['queries'].tolist()))

queries = queries_file['queries'].tolist()
query_ids = queries_file['qid'].tolist()


In [ ]:

path = '/Users/user/question-retrieval-KIPerWeb/runs/BM25/json_format/'


for i, j in zip(queries, query_ids):
    BM25_ranker(i, qb, k=100, run_path=f'{path}query_{j}/run_{j}.json')


In [ ]:
path = '/Users/user/question-retrieval-KIPerWeb/runs/BM25/'

json_files = list_files_from_dir(path, extention='.json')
json_files

In [ ]:
# For each JSON output, transform the results to a TREC format

for e in json_files:
    try:
        to_trec_format(e)
    except IndexError:
        print(f"Error! in {e}")

In [ ]:
list_files_from_dir(path, extention='.tsv')

In [ ]:

path_list_processed = list_files_from_dir(path, extention='.tsv')

list_of_dfs = []
for e in path_list_processed:
    df = pd.concat([pd.read_csv(e, header=None, sep='\t', names=["queries", "Q0", "docid", "rank", "score", "tag"])])
    list_of_dfs.append(df)
    


all_runs = pd.concat(list_of_dfs).reset_index(drop=True)
all_runs = pd.merge(queries_file, all_runs, on='queries', how='inner')[['qid', 'Q0', 'docid', 'rank', 'score','tag']]
all_runs['qid'] = all_runs['qid'].astype(str)
all_runs['docid'] = all_runs['docid'].astype(str)
all_runs

In [ ]:
L_models = list(set(all_runs['tag']))
runs = [all_runs.query(f'tag=="{e}"') for e in L_models]

for i, j in zip(runs, L_models):
    i.to_csv(f'/Users/user/question-retrieval-KIPerWeb/runs/BM25/trec_format/{j}_run.txt', header = None, sep='\t',index=False )
    

In [ ]:
# from ranx import Qrels, evaluate, Run

bm25_run = Run.from_file("/Users/user/question-retrieval-KIPerWeb/runs/BM25/trec_format/BM25_run.txt")
# bm25_run

In [ ]:

f = "/Users/user/question-retrieval-KIPerWeb/testbeds/queries_experiments/trec_pools/testbed.csv"

qrels_df = pd.read_csv(f)
qrels_df['qid'] = qrels_df['qid'].astype(str)
qrels_df['docid'] = qrels_df['docid'].astype(str)

qrels = Qrels.from_df(
    df=qrels_df,
    q_id_col="qid",
    doc_id_col="docid",
    score_col="qrel",
)


In [ ]:
results = []
runs = [bm25_run]
runs_names = ['bm25']


report = create_report(qrels, runs,
                       runs_names, 25,"/Users/user/question-retrieval-KIPerWeb/data/results/bm25/bm25_at_25.csv")

report.sort_values(by='mrr@25',  ascending=False)

In [ ]:
report = create_report(qrels, runs, 
                       runs_names, 50,"/Users/user/question-retrieval-KIPerWeb/data/results/bm25/bm25_at_50.csv")

report.sort_values(by='mrr@50',  ascending=False)

In [ ]:
report = create_report(qrels, runs, 
                       runs_names, 100,"/Users/user/question-retrieval-KIPerWeb/data/results/bm25/bm25_at_100.csv")

report.sort_values(by='mrr@100',  ascending=False)